In [242]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import sys

from fast_ml.model_development import train_valid_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.svm import SVR

from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

from sklearn import metrics 
from sklearn import tree

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier


### Split data by browser and platform_os, and version each split as a new version of the data in dvc.

In [243]:
df_split = pd.read_csv('../data/data_with_response.csv')
df_split.head()

,auction_id,experiment,date,hour,device_make,platform_os,browser,user_response
0,008aafdf-deef-4482-8fec-d98e3da054da,exposed,2020-07-04,16,Generic Smartphone,6,Chrome Mobile,1
1,00b6fadb-10bd-49e3-a778-290da82f7a8d,control,2020-07-08,4,Samsung SM-A202F,6,Facebook,1
2,018af862-486e-4da1-a85b-71872120e57c,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,1
3,023ec7b5-cb8f-49a5-995f-e0d7c2f702e5,exposed,2020-07-09,13,Samsung SM-G935F,6,Facebook,1
4,02efdb70-8596-4f3f-b0b2-b91e194f61f7,exposed,2020-07-05,6,Generic Smartphone,6,Chrome Mobile,1


In [244]:
label_encoder = LabelEncoder()
# 
df_split['experiment'] = label_encoder.fit_transform(df_split['experiment'])
df_split['date'] = label_encoder.fit_transform(df_split['date'])
df_split['hour'] = label_encoder.fit_transform(df_split['hour'])
df_split['device_make'] = label_encoder.fit_transform(df_split['device_make'])
df_split['platform_os'] = label_encoder.fit_transform(df_split['platform_os'])
df_split['browser'] = label_encoder.fit_transform(df_split['browser'])
df_split['user_response'] = label_encoder.fit_transform(df_split['user_response'])

#### Take Data With Browsers and Platforms

In [245]:
df_Platform_copied = df_split.copy()
df_platform = df_Platform_copied.drop(columns='browser', axis=1)
df_platform.head()

,auction_id,experiment,date,hour,device_make,platform_os,user_response
0,008aafdf-deef-4482-8fec-d98e3da054da,1,1,16,13,1,1
1,00b6fadb-10bd-49e3-a778-290da82f7a8d,0,5,4,43,1,1
2,018af862-486e-4da1-a85b-71872120e57c,0,0,15,13,1,1
3,023ec7b5-cb8f-49a5-995f-e0d7c2f702e5,1,6,13,65,1,1
4,02efdb70-8596-4f3f-b0b2-b91e194f61f7,1,2,6,13,1,1


In [246]:
df_platform.to_csv('../data/data_with_platform.csv', index=False)

In [247]:
df_os_copied = df_split.copy()
df_os = df_os_copied.drop(columns='platform_os', axis=1)
df_os.head()

,auction_id,experiment,date,hour,device_make,browser,user_response
0,008aafdf-deef-4482-8fec-d98e3da054da,1,1,16,13,1,1
1,00b6fadb-10bd-49e3-a778-290da82f7a8d,0,5,4,43,4,1
2,018af862-486e-4da1-a85b-71872120e57c,0,0,15,13,1,1
3,023ec7b5-cb8f-49a5-995f-e0d7c2f702e5,1,6,13,65,4,1
4,02efdb70-8596-4f3f-b0b2-b91e194f61f7,1,2,6,13,1,1


In [248]:
df_os.to_csv('../data/data_with_os.csv', index=False)

### Access Data From DVC

In [249]:

import dvc.api

with dvc.api.open(
    'data/data_with_os.csv',
    mode='rb',
    
) as data:
    df_with_browser = pd.read_csv(data)
    df_with_browser.head()
df_with_browser.drop(['auction_id'],axis=1,inplace=True)

# Logistic Regression

### For Data Version With Platform Os

In [250]:
df_with_browser.head()

,experiment,date,hour,device_make,browser,user_response
0,1,1,16,13,1,1
1,0,5,4,43,4,1
2,0,0,15,13,1,1
3,1,6,13,65,4,1
4,1,2,6,13,1,1


In [251]:
df_with_browser.shape

(1243, 6)

In [252]:
X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(df_with_browser, target = 'user_response', 
                                                                            train_size=0.7, valid_size=0.2, test_size=0.1)
print(X_train.shape), print(y_train.shape)
print(X_valid.shape), print(y_valid.shape)
print(X_test.shape), print(y_test.shape)

(870, 5)
(870,)
(248, 5)
(248,)
(125, 5)
(125,)


(None, None)

### Prepare cross-validation procedure

In [253]:

cv = RepeatedKFold(n_splits=10, n_repeats=5, random_state=1)


### Create Model

### Model Evaluation

In [254]:
# evaluate model
# scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)


model = LogisticRegression()
model.fit(X_train, y_train)
result=cross_val_score(estimator=model,X=X_train,y=y_train,cv=5,scoring='accuracy')

print(result)
print('Mean Logistic Regression score for browser data :',result.mean())



[0.59195402 0.55172414 0.48850575 0.53448276 0.5       ]
Mean Logistic Regression score for browser data : 0.5333333333333333


### Accuracy: 0.708046

### For Data Version With Browser

In [255]:
with dvc.api.open(
    'data/data_with_platform.csv',
    mode='rb',
    
) as data:
    df_with_platform_os = pd.read_csv(data)
    df_with_platform_os.head()
df_with_platform_os.drop(['auction_id'],axis=1,inplace=True)

In [256]:
df_with_platform_os.head()

,experiment,date,hour,device_make,platform_os,user_response
0,1,1,16,13,1,1
1,0,5,4,43,1,1
2,0,0,15,13,1,1
3,1,6,13,65,1,1
4,1,2,6,13,1,1


In [258]:
X_train2, y_train2, X_valid2, y_valid2, X_test2, y_test2 = train_valid_test_split(df_with_platform_os, target = 'user_response', 
                                                                            train_size=0.7,valid_size=0.2, test_size=0.1)


print(X_train2.shape), print(y_train2.shape)
print(X_valid2.shape), print(y_valid2.shape)
print(X_test2.shape), print(y_test2.shape)

(870, 5)
(870,)
(248, 5)
(248,)
(125, 5)
(125,)


(None, None)

In [259]:

cv2 = RepeatedKFold(n_splits=10, n_repeats=5, random_state=1)


In [275]:
# evaluate model
# scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)


model2 = LogisticRegression()
model2.fit(X_train2, y_train2)
result2=cross_val_score(estimator=model2,X=X_train2,y=y_train2,cv=5,scoring='accuracy')

print(result2)
print('Mean Logistic Regression score for Browser data :',result2.mean())



[0.52873563 0.51724138 0.55172414 0.55172414 0.54022989]
Mean Logistic Regression score for Browser data : 0.5379310344827586


### Accuracy: 0.9988506

# Decision Tree

### For Data Version With Platform Os

In [273]:

clf = tree.DecisionTreeClassifier()
decision_tree_model = clf.fit(X_train, y_train)

scores = cross_val_score(estimator=clf, X=X_train, y=y_train, cv=5, scoring='accuracy')

print(scores)
print('Mean Logistic Regression score for platform os data :',scores.mean())



[0.53448276 0.57471264 0.50574713 0.52873563 0.52298851]
Mean Logistic Regression score for platform os data : 0.5333333333333334


### Accuracy: 0.7

### For Data Version With Browser

In [262]:

clf2 = tree.DecisionTreeClassifier()
decision_tree_model = clf2.fit(X_train2, y_train2)

scores2 = cross_val_score(estimator=clf2, X=X_train2, y=y_train2, cv=5, scoring='accuracy')

print(scores2)
print('Mean Logistic Regression score for platform os data :',scores2.mean())


[0.49425287 0.55172414 0.51724138 0.48850575 0.48850575]
Mean Logistic Regression score for platform os data : 0.5080459770114942


### Accuracy: 0.9988

# XGBoost

### For Data Version With Platform Os

In [263]:
df_with_platform_os.head()

,experiment,date,hour,device_make,platform_os,user_response
0,1,1,16,13,1,1
1,0,5,4,43,1,1
2,0,0,15,13,1,1
3,1,6,13,65,1,1
4,1,2,6,13,1,1


In [264]:

xgbs = XGBClassifier(n_estimators=1000, learning_rate=0.1)

xgbs.fit(
    X_train, y_train,
    early_stopping_rounds=5,
    eval_set=[(X_test, y_test)],
    verbose=False
 )

scores_xgb = cross_val_score(estimator=xgbs, X=X_train, y=y_train, cv=5, scoring='accuracy')

print(scores_xgb)
print('Mean Logistic Regression score for platform os data :',scores_xgb.mean())



[0.48850575 0.51149425 0.45977011 0.55172414 0.54022989]
Mean Logistic Regression score for platform os data : 0.5103448275862068


### For Data Version With Browser

In [265]:

xgbs2 = XGBClassifier(n_estimators=1000, learning_rate=0.1)

xgbs2.fit(
    X_train2, y_train2,
    early_stopping_rounds=5,
    eval_set=[(X_test2, y_test2)],
    verbose=False
 )

scores_xgb2 = cross_val_score(estimator=xgbs2, X=X_train2, y=y_train2, cv=5, scoring='accuracy')

print(scores_xgb2)
print('Mean Logistic Regression score for platform os data :',scores_xgb2.mean())



[0.5        0.55172414 0.56896552 0.50574713 0.5       ]
Mean Logistic Regression score for platform os data : 0.525287356321839


### Accuracy: 0.9977

# Random Forest

### For Browser Data

In [266]:
clf_rf = RandomForestClassifier(n_estimators = 100)
clf_rf.fit(X_train, y_train)

result_rf=cross_val_score(estimator=clf_rf,X=X_train,y=y_train,cv=5,scoring='accuracy')


print(result_rf)
print('Mean Logistic Regression score for platform os data :',result_rf.mean())

[0.51149425 0.57471264 0.4137931  0.55747126 0.51724138]
Mean Logistic Regression score for platform os data : 0.5149425287356322


#### Accuracy: 0.708046

### For Platform Os Data

In [267]:
clf_rf2 = RandomForestClassifier(n_estimators = 100)
clf_rf2.fit(X_train2, y_train2)

result_rf2=cross_val_score(estimator=clf_rf2,X=X_train2,y=y_train2,cv=5,scoring='accuracy')


print(result_rf2)
print('Mean Logistic Regression score for platform os data :',result_rf2.mean())

[0.51724138 0.56896552 0.55172414 0.51149425 0.50574713]
Mean Logistic Regression score for platform os data : 0.5310344827586206


#### Accuracy: 0.998851

In [268]:
# Accuracy Scores
# First parameter is actual value
# second parameter is value that we prediceted

# Logistics Regression

# Decision Tree

#XGBoost

# Random Forest

print("***********************")
print("Cross Validation Score")
print("***********************")

print("Logistic Regression For Browser Data:", result.mean())
print("Logistic Regression:", result2.mean())
print(' ')
print("Desicion Tree For Browser Data:", scores.mean())
print("Desicion Tree:", scores2.mean())
print(' ')

print("XGBoost For Browser Data:", scores_xgb.mean())
print("XGBoost:", scores_xgb2.mean())
print(' ')

print("Random Forest For Browser Data:", result_rf.mean())
print("Random Forest:", result_rf2.mean())


***********************
Cross Validation Score
***********************
Logistic Regression For Browser Data: 0.5333333333333333
Logistic Regression: 0.5379310344827586
 
Desicion Tree For Browser Data: 0.525287356321839
Desicion Tree: 0.5080459770114942
 
XGBoost For Browser Data: 0.5103448275862068
XGBoost: 0.525287356321839
 
Random Forest For Browser Data: 0.5149425287356322
Random Forest: 0.5310344827586206


# Compute feature importance

## Logistics Regression

#### For Browser Data

In [269]:
logistic_imp_feature = pd.DataFrame({"Feature Importance": model.coef_[0]})
logistic_imp_feature["Feature"] = ['experiment'	,'date'	,'hour','device_make',	'browser']
logistic_imp_feature = logistic_imp_feature.set_index("Feature")
logistic_imp_feature = logistic_imp_feature.sort_values(by=["Feature Importance"], ascending=False)
logistic_imp_feature

,Feature Importance
Feature,
hour,0.017556
device_make,0.003557
experiment,-0.015088
date,-0.018042
browser,-0.060585


#### Important Feature: hour

#### For Platform Os Data

In [276]:
logistic_imp_feature_os = pd.DataFrame({"Feature Importance": model2.coef_[0]})
logistic_imp_feature_os["Feature"] = ['experiment'	,'date'	,'hour','device_make',	'platform_os']
logistic_imp_feature_os = logistic_imp_feature_os.set_index("Feature")
logistic_imp_feature_os = logistic_imp_feature_os.sort_values(by=["Feature Importance"], ascending=False)
logistic_imp_feature_os

,Feature Importance
Feature,
platform_os,0.537571
experiment,0.147663
hour,0.009032
date,0.006679
device_make,0.004193


Important Feature: platform_os

## Decision Tree

#### For Browser Data

In [278]:
decision_imp_feature_browser = pd.DataFrame({"Feature Importance": clf.feature_importances_})
decision_imp_feature_browser["Feature"] = ["experiment", "date", "hour", "device_make", "browser"]
decision_imp_feature_browser = decision_imp_feature_browser.set_index("Feature")
decision_imp_feature_browser = decision_imp_feature_browser.sort_values(by=["Feature Importance"], ascending=False)
decision_imp_feature_browser

,Feature Importance
Feature,
hour,0.316341
date,0.264920
device_make,0.215436
experiment,0.105644
browser,0.097659


#### Important Feature: hour

### For Platform Os Data

In [279]:
decision_imp_feature_platform = pd.DataFrame({"Feature Importance": clf2.feature_importances_})
decision_imp_feature_platform["Feature"] = ["experiment", "date", "hour", "device_make", "platform_os"]
decision_imp_feature_platform = decision_imp_feature_platform.set_index("Feature")
decision_imp_feature_platform = decision_imp_feature_platform.sort_values(by=["Feature Importance"], ascending=False)
decision_imp_feature_platform

,Feature Importance
Feature,
hour,0.392633
device_make,0.309584
date,0.221603
experiment,0.072415
platform_os,0.003765


#### Important Feature: hour

## XGBoost

### For Browser Data

In [285]:
# xgbs2.feature_importances_
xgb_imp_feature_browser = pd.DataFrame({"Feature Importance": xgbs2.feature_importances_})
xgb_imp_feature_browser["Feature"] = ["experiment", "date", "hour", "device_make", "browser"]
xgb_imp_feature_browser = xgb_imp_feature_browser.set_index("Feature")
xgb_imp_feature_browser = xgb_imp_feature_browser.sort_values(by=["Feature Importance"], ascending=False)
xgb_imp_feature_browser

,Feature Importance
Feature,
device_make,0.302024
hour,0.279457
date,0.272501
experiment,0.146018
browser,0.000000


#### Important Feature: device_make

### For Platform Os Data

In [286]:
# xgbs2.feature_importances_
xgb_imp_feature_platform = pd.DataFrame({"Feature Importance": xgbs2.feature_importances_})
xgb_imp_feature_platform["Feature"] = ["experiment", "date", "hour", "device_make", "platform_os"]
xgb_imp_feature_platform = xgb_imp_feature_platform.set_index("Feature")
xgb_imp_feature_platform = xgb_imp_feature_platform.sort_values(by=["Feature Importance"], ascending=False)
xgb_imp_feature_platform

,Feature Importance
Feature,
device_make,0.302024
hour,0.279457
date,0.272501
experiment,0.146018
platform_os,0.000000


#### Important Feature: device_make

# Random Forest

### For Browser Data

In [287]:
rand_imp_feature_browser = pd.DataFrame({"Feature Importance": clf_rf.feature_importances_})
rand_imp_feature_browser["Feature"] = ["experiment", "date", "hour", "device_make", "browser"]
rand_imp_feature_browser = rand_imp_feature_browser.set_index("Feature")
rand_imp_feature_browser = rand_imp_feature_browser.sort_values(by=["Feature Importance"], ascending=False)
rand_imp_feature_browser

,Feature Importance
Feature,
hour,0.411347
device_make,0.259540
date,0.188380
browser,0.083682
experiment,0.057051


#### Important Feature: hour

### For Platform Os Data

In [288]:
rand_imp_feature_platform = pd.DataFrame({"Feature Importance": clf_rf.feature_importances_})
rand_imp_feature_platform["Feature"] = ["experiment", "date", "hour", "device_make", "platform_od"]
rand_imp_feature_platform = rand_imp_feature_platform.set_index("Feature")
rand_imp_feature_platform = rand_imp_feature_platform.sort_values(by=["Feature Importance"], ascending=False)
rand_imp_feature_platform

,Feature Importance
Feature,
hour,0.411347
device_make,0.259540
date,0.188380
platform_od,0.083682
experiment,0.057051


#### Important Feature: hour